<img style="float: right;" src="https://cdn.bulbagarden.net/upload/e/e2/133Eevee.png" width="150" height="150" />
# SQL with Pokemon - Chapter 2



Welcome to the world of Pokemon! Most people who come through here are looking to start their Pokemon journey by catching, training and discovering Pokemon. It is important for people to go out into the unknown and discover secrets about Pokemon that we have yet to unravel. 

However, there's another option you may take, one that is equally as important, if not more important. You could become a Pokemon Researcher and help construct our Pokemon Database! Trainers out in the field need to have a wealth of accurate information in order for them to make informed decisions on their journey. You will still get the chance interact with Pokemon on a daily basis, but your main role is to work with our new SQL technology to create data tables that will best inform our trainers. Your work here in the Research Lab will affect the lives of thousands, maybe even millions of people! 

Does that sound like a journey worth embarking on? When you are ready, press START!

# Before we begin

NOTE: I have added Abilites.xlsx to my SQL Server Pokemon Database so we can demonstrate JOINs, UNIONs and Subqueries

In [1]:
import numpy as np
import pandas as pd
import pyodbc

conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=localhost;"
                      "Database=Pokemon;"
                      "Trusted_Connection=yes;")
cursor = conn.cursor()

<img style="float: right;" src="https://cdn.bulbagarden.net/upload/5/53/054Psyduck.png" width="150" height="150" />
# What is joining tables? 


To put it simply, SQL has the ability to "join" two tables together when they share values in common. There are 3 main types of joins: Cross Joins, Inner Joins and Outer Joins. I think an example would help explain this better.

The abilities table contains the Pokemon abilities absent from our main Pokemon data table. So let's join them together!

In [6]:
# First we do a Cross Join, and...something doesn't look right
df = pd.read_sql_query('''
SELECT
    p.[Name],
    [Type 1],
    [Type 2],
    [Ability 1],
    [Ability 2],
    [Generation]
FROM pokemon AS p
CROSS JOIN abilities AS a
''', conn)

df.head(10)

,Name,Type 1,Type 2,Ability 1,Ability 2,Generation
0,Bulbasaur,Grass,Poison,Overgrow,-----,1.0
1,Ivysaur,Grass,Poison,Overgrow,-----,1.0
2,Venusaur,Grass,Poison,Overgrow,-----,1.0
3,VenusaurMega Venusaur,Grass,Poison,Overgrow,-----,1.0
4,Charmander,Fire,None,Overgrow,-----,1.0
5,Charmeleon,Fire,None,Overgrow,-----,1.0
6,Charizard,Fire,Flying,Overgrow,-----,1.0
7,CharizardMega Charizard X,Fire,Dragon,Overgrow,-----,1.0
8,CharizardMega Charizard Y,Fire,Flying,Overgrow,-----,1.0
9,Squirtle,Water,None,Overgrow,-----,1.0


That's not right! We all know that Charmander and Squirtle don't have Overgrow as an Ability! What gives!?

What Cross Joins does is that it creates a Cartesian Product between the two. So if you have m rows in table 1 and n rows in table 2, the result set is m x n rows. 

Cross Joins do have their uses in other areas of SQL and Databases, but we need a different kind of join in order to get what we want. So let's try out an Inner Join!

In [10]:
# First we do a Cross Join, and...something doesn't look right
df = pd.read_sql_query('''
SELECT
    p.[Name],
    [Type 1],
    [Type 2],
    [Ability 1],
    [Ability 2],
    [Generation]
FROM pokemon AS p
INNER JOIN abilities AS a
ON p.[Name] = a.[Name]
WHERE a.[Form] <> 'Mega'
''', conn)

df.head(10)

,Name,Type 1,Type 2,Ability 1,Ability 2,Generation
0,Abomasnow,Grass,Ice,Snow Warning,-----,4.0
1,Abra,Psychic,None,Synchronize,Inner Focus,1.0
2,Absol,Dark,None,Pressure,Super Luck,3.0
3,Accelgor,Bug,None,Hydration,Sticky Hold,5.0
4,Aerodactyl,Rock,Flying,Rock Head,Pressure,1.0
5,Aggron,Steel,Rock,Sturdy,Rock Head,3.0
6,Aipom,Normal,None,Run Away,Pickup,2.0
7,Alakazam,Psychic,None,Synchronize,Inner Focus,1.0
8,Alomomola,Water,None,Healer,Hydration,5.0
9,Altaria,Dragon,Flying,Natural Cure,-----,3.0


That's much better! Instead of creating a Cartesian Product, we "joined" the Names from Pokemon and the Names from Abilities, and that allows us to Query the Ability 1 and Ability 2 columns. Now our Pokemon will have their Types, Base Stats and Abilities to go with them!

<img style="float: right;" src="https://cdn.bulbagarden.net/upload/4/41/373Salamence.png" width="150" height="150" />
# Situation 1 - Intimidated?


You are getting the rare opportunity to train with Bruno from the Elite Four! He uses mainly Fighting Type Pokemon, with high Attack power. We need Pokemon to be able to handle his team.

With our newfound ability to join tables, we can filter which Pokemon have the Ability Intimidate, which lowers the Attack power of the other Pokemon when it is sent into battle.

In [16]:
df = pd.read_sql_query('''
SELECT
    p.[Name],
    [Type 1],
    [Type 2],
    [Ability 1],
    [Ability 2],
    [Generation]
FROM pokemon AS p
INNER JOIN abilities AS a
ON p.[Name] = a.[Name]
WHERE a.[Form] <> 'Mega'
AND [Ability 1] = 'Intimidate'
OR [Ability 2] = 'Intimidate'
''', conn)

df

,Name,Type 1,Type 2,Ability 1,Ability 2,Generation
0,Ekans,Poison,None,Intimidate,Shed Skin,1.0
1,Arbok,Poison,None,Intimidate,Shed Skin,1.0
2,Growlithe,Fire,None,Intimidate,Flash Fire,1.0
3,Arcanine,Fire,None,Intimidate,Flash Fire,1.0
4,Tauros,Normal,None,Intimidate,Anger Point,1.0
5,Gyarados,Water,Flying,Intimidate,-----,1.0
6,Snubbull,Fairy,None,Intimidate,Run Away,2.0
7,Granbull,Fairy,None,Intimidate,Quick Feet,2.0
8,Stantler,Normal,None,Intimidate,Frisk,2.0
9,Hitmontop,Fighting,None,Intimidate,Technician,2.0


Looking at our list, we see several good choices. Salamence and Gyarados would be definitely good picks against Bruno's team. The Flying type adavantage would make it difficult for him to do good damage to us. Arcanine wouldn't be too bad either. We are probably going to have to pick Pokemon outside this, because most of the other Pokemon with Intimidate are either Dark or Normal types, which get pummeled by Fighting types.

<img style="float: right;" src="https://archives.bulbagarden.net/media/upload/f/fa/257Blaziken-Mega.png" width="200" height="200" />
# Situation 2 - One on One Showdown!


Your rival, Blue, wants to challenge you to a One on One Pokemon Battle! He will use Mega Blaziken as his Pokemon of choice. Mega Blaziken is outrageously powerful, so using any Pokemon that are weak against it would be suicide. We need to pick a Pokemon that can do good damage while also having enough defense to tank hits.

In [32]:
df = pd.read_sql_query('''
SELECT
    p.[Name],
    [Type 1],
    [Type 2],
    [Ability 1],
    [Ability 2],
    [HP],
    [Attack],
    [Defense],
    [Special Attack],
    [Special Defense],
    [Speed],
    [Generation]
FROM pokemon AS p
INNER JOIN abilities AS a
ON p.[Name] = a.[Name]
WHERE [Type 1] IN ('Water', 'Psychic', 'Dragon')
AND [Defense] > 80
AND [Attack] > 80
AND [Legendary] = 0
OR [Type 2] IN ('Water', 'Psychic', 'Dragon')
AND [Defense] > 80
AND [Attack] > 80
AND [Legendary] = 0
''', conn)

df.head(100)

,Name,Type 1,Type 2,Ability 1,Ability 2,HP,Attack,Defense,Special Attack,Special Defense,Speed,Generation
0,Barbaracle,Rock,Water,Tough Claws,Sniper,72.0,105.0,115.0,54.0,86.0,68.0,6.0
1,Blastoise,Water,None,Torrent,-----,79.0,83.0,100.0,85.0,105.0,78.0,1.0
2,Blastoise,Water,None,Mega Launcher,-----,79.0,83.0,100.0,85.0,105.0,78.0,1.0
3,Bronzong,Steel,Psychic,Levitate,Heatproof,67.0,89.0,116.0,79.0,116.0,33.0,4.0
4,Carracosta,Water,Rock,Solid Rock,Sturdy,74.0,108.0,133.0,83.0,65.0,32.0,5.0
5,Celebi,Psychic,Grass,Natural Cure,-----,100.0,100.0,100.0,100.0,100.0,100.0,2.0
6,Cloyster,Water,Ice,Shell Armor,Skill Link,50.0,95.0,180.0,85.0,45.0,70.0,1.0
7,Crawdaunt,Water,Dark,Hyper Cutter,Shell Armor,63.0,120.0,85.0,90.0,55.0,55.0,3.0
8,Dragonite,Dragon,Flying,Inner Focus,-----,91.0,134.0,95.0,100.0,100.0,80.0,1.0
9,Druddigon,Dragon,None,Rough Skin,Sheer Force,77.0,120.0,90.0,60.0,90.0,48.0,5.0


A lot of considerations here. We went with Base Stats for Attack and Defense above 80 since Mega Blaziken is going to hit really hard. Swampert wouldn't be too bad with his big HP and good tanking stats. Garchomp and its Sand Force ability could allow us to hit his Mega Blaziken harder. Dragonite would be a good pick also.

# What is a Subquery?

Now we are getting to the cool stuff. Subqueries allow us to nest queries inside queries. Let's look at an example:

In [42]:
df = pd.read_sql_query('''
SELECT
    [Name],
    [Type 1],
    [Type 2],
    [HP],
    [Attack],
    [Defense],
    [Special Attack],
    [Special Defense],
    [Speed]
FROM pokemon
WHERE [Special Attack] > (
    SELECT
        AVG([Special Attack])
    FROM pokemon
    WHERE [Speed] > 50
    AND [Type 1] = 'Water'
)
AND [Defense] > 135
''', conn)

df.head(100)

,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed
0,SlowbroMega Slowbro,Water,Psychic,95.0,75.0,180.0,130.0,80.0,30.0
1,Cloyster,Water,Ice,50.0,95.0,180.0,85.0,45.0,70.0
2,TyranitarMega Tyranitar,Rock,Dark,100.0,164.0,150.0,95.0,120.0,71.0
3,Torkoal,Fire,None,70.0,85.0,140.0,85.0,70.0,20.0
4,MetagrossMega Metagross,Steel,Psychic,80.0,145.0,150.0,105.0,110.0,110.0
5,Groudon,Ground,None,100.0,150.0,140.0,100.0,90.0,90.0
6,GroudonPrimal Groudon,Ground,Fire,100.0,180.0,160.0,150.0,90.0,90.0
7,Cofagrigus,Ghost,None,58.0,50.0,145.0,95.0,105.0,30.0
8,Diancie,Rock,Fairy,50.0,100.0,150.0,100.0,150.0,50.0


So the logic for this query starts with the WHERE statement. The WHERE Statement is saying, "I want to find the Special Attack that is higher than the Average Special Attack of Pokemon with a Speed base Stat of 50 AND are Water Types". After that, we do another where statement on Pokemon with a Defense Base Stat of over 135.

Diving into all of the use-cases of Subqueries is outside of the scope of this Juypter Notebook. Just be aware that Subqueries will allow you to perform logic on Data Tables in creative ways, while maintaining readability.

<img style="float: right;" src="https://cdn.bulbagarden.net/upload/6/65/HeartGold_SoulSilver_Sabrina.png" width="100" height="100" />
<img style="float: right;" src="https://cdn.bulbagarden.net/upload/4/4a/HeartGold_SoulSilver_Erika.png" width="60" height="60" />
# Situation 3 - Gym Leader Double Defense



Gym Leaders Erika and Sabrina have teamed up to ward off Giovanni from taking over Saffron City! They will each carry 3 Pokemon from their respective rosters, and team up to fight against Team Rocket!

In [45]:
df = pd.read_sql_query('''
SELECT 
	[Name],
	[Type 1],
	[Type 2],
	[Total],
	[HP],
	[Attack],
	[Defense],
	[Special Attack],
	[Special Defense],
	[Speed]
FROM [Pokemon].[dbo].[pokemon]
WHERE [Type 1] = 'Psychic'
AND Legendary = 0
AND [Name] NOT LIKE '%Mega%'
AND [Special Attack] > 120
AND [Speed] > 70

UNION ALL

SELECT 
	[Name],
	[Type 1],
	[Type 2],
	[Total],
	[HP],
	[Attack],
	[Defense],
	[Special Attack],
	[Special Defense],
	[Speed]
FROM [Pokemon].[dbo].[pokemon]
WHERE [Type 1] = 'Grass'
AND Legendary = 0
AND [Name] NOT LIKE '%Mega%'
AND [Special Defense] > 90
AND [Defense] > 90
AND [HP] > 70
''', conn)

df.head(100)

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed
0,Alakazam,Psychic,None,500.0,55.0,50.0,45.0,135.0,95.0,120.0
1,Espeon,Psychic,None,525.0,65.0,65.0,60.0,130.0,95.0,110.0
2,Gardevoir,Psychic,Fairy,518.0,68.0,65.0,65.0,125.0,115.0,80.0
3,Bellossom,Grass,None,490.0,75.0,80.0,95.0,90.0,100.0,50.0
4,Serperior,Grass,None,528.0,75.0,75.0,95.0,75.0,95.0,113.0
5,Ferrothorn,Grass,Steel,489.0,74.0,94.0,131.0,54.0,116.0,20.0


We introduce UNIONS for this Situation. UNIONS allow us to combine two sets of SELECT statements as long as the columns are the same in both of them. In this scenario, I used multiple WHERE statements to construct a 6 Party team for Sabrina (Psychic Types) and Erika (Grass Types).

<img style="float: right;" src="https://cdn.bulbagarden.net/upload/8/8f/HeartGold_SoulSilver_Lt_Surge.png" width="75" height="75" />
# Situation 4 - Lt. Surge promoted?


Major General Olivier Mira Armstrong has asked Lt. Surge to come out of retirement to defend The Northern Wall of Briggs. Armstrong realizes Surge's leadership abilities, and promoted him to Major for the upcoming battle. Surge has asked you to take over the gym while he's away, but you have to use his Pokemon as part of the deal.

In [46]:
df = pd.read_sql_query('''
WITH surge AS (
SELECT
	[Name],
	[Type 1],
	[Type 2],
	[HP],
	[Attack],
	[Defense],
	[Special Attack],
	[Special Defense],
	[Speed]
FROM [Pokemon].[dbo].[pokemon]
WHERE [Type 1] = 'Electric'
AND Legendary = 0
AND [Name] NOT LIKE '%Mega%'
)

SELECT *
FROM surge
WHERE [Speed] > 120
AND [Special Attack] > 100

UNION ALL

SELECT *
FROM surge
WHERE [Defense] > 100
AND [Special Attack] > 120

UNION ALL

SELECT *
FROM surge
WHERE [Attack] > 100
AND [Speed] > 90
''', conn)

df.head(100)

,Name,Type 1,Type 2,HP,Attack,Defense,Special Attack,Special Defense,Speed
0,Jolteon,Electric,None,65.0,65.0,60.0,110.0,95.0,130.0
1,Magnezone,Electric,Steel,70.0,70.0,115.0,130.0,90.0,60.0
2,Electivire,Electric,None,75.0,123.0,67.0,95.0,85.0,95.0


Combining UNIONS with Common Table Expressions (CTE) we can create multiple SELECT statements while keeping readability. The CTE at the top we define as "surge" and we get the base stats, the name and we have a WHERE statement on Electric types. After that, we can use surge as a temporary table to query off of. We then have 3 seperate SELECT statements linked together with UNION ALLs in order to get our 3 Pokemon team!